In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from datasets import load_dataset

from matplotlib import pyplot as plt
%matplotlib inline

# Download census-income dataset

In [3]:
data_files = {"train": "train.csv", "validation": "validation.csv", "test":"test.csv"}
dataset = load_dataset("PDJ107/riot-data", data_files=data_files, revision='cgm_20')

Using custom data configuration PDJ107--riot-data-361ab84a17fdf47e
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/PDJ107--riot-data-361ab84a17fdf47e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train = dataset['train'].to_pandas()
train = train.drop(['_matchId'], axis=1)
train['Set'] = 'train'
valid = dataset['validation'].to_pandas()
valid = valid.drop(['_matchId'], axis=1)
valid['Set'] = 'valid'
test = dataset['test'].to_pandas()
test = test.drop(['_matchId'], axis=1)
test['Set'] = 'test'

train = pd.concat([train, valid, test])

In [5]:
train

,totalMinionsKilled_0_s,wardsPlaced_0_s,damageDealtToObjectives_0_s,turretKills_0_s,physicalDamageTaken_0_s,totalDamageDealt_0_s,assists_0_s,damageDealtToBuildings_0_s,neutralMinionsKilled_0_s,physicalDamageDealtToChampions_0_s,...,magicDamageDealt_9_s,damageSelfMitigated_9_s,damageDealtToTurrets_9_s,turretsLost_9_s,totalTimeCCDealt_9_s,totalDamageTaken_9_s,visionWardsBoughtInGame_9_s,physicalDamageDealt_9_s,win,Set
0,7.384349,0.184237,545.298076,0.124783,817.139596,6064.634538,0.178361,219.112071,1.284994,794.427927,...,663.584754,324.566879,76.928818,0.102803,10.181232,433.579509,0.153529,178.532931,0,train
1,6.211681,0.189883,234.133063,0.041398,323.025455,3713.142398,0.251829,92.548929,0.276442,517.771145,...,365.601654,738.315027,13.437196,0.224642,1.839656,529.999910,0.137576,151.173466,0,train
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,train
3,6.494377,0.323323,191.637547,0.038838,366.956800,3450.909061,0.221483,117.183927,0.108090,126.070478,...,292.696414,523.134053,27.740439,0.152885,2.118536,516.618493,0.380332,207.602800,1,train
4,4.901070,0.408351,202.392028,0.043064,682.581669,3387.382600,0.352387,167.950305,0.218166,492.481169,...,1508.075176,1011.594398,34.129517,0.222025,3.620328,905.347510,0.069698,71.321709,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57446,6.556485,0.351607,250.857537,0.048551,653.756544,3793.560072,0.148246,161.581983,0.228864,383.499664,...,265.278741,209.247839,34.297628,0.183640,6.281197,399.385287,0.202918,177.018736,1,test
57447,5.561225,0.365998,232.046061,0.034106,501.187852,3331.827664,0.229512,119.643454,0.024977,437.746144,...,339.901641,618.165555,12.523118,0.193511,7.876048,451.764455,0.335950,288.089896,0,test
57448,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,845.495514,207.942847,20.437388,0.172670,5.299503,526.386009,0.244624,122.823517,0,test
57449,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,test


In [6]:
# dataset_name = 'train'
# out = Path(os.getcwd()+'/data/'+dataset_name+'.csv')

# label_name = 'LABEL_mastery_and_win_rate'
# label_path = Path(os.getcwd()+'/data/'+label_name+'.csv')

# test_dataset_name = 'test'
# test_out = Path(os.getcwd()+'/data/'+test_dataset_name+'.csv')

# Load data and split

In [7]:
# with open(label_path, 'r', encoding='UTF8') as f:
#     raw_labels = f.readlines()

# label_dict={}
# win_idx = raw_labels[0].split(',').index('_win_0_l')
# matchid_idx = raw_labels[0].split(',').index('_matchId\n')
# for raw_label in raw_labels[1:]:
#     raw_label = raw_label.split(',')
#     label_dict[raw_label[matchid_idx].strip()] = int(raw_label[win_idx])

In [8]:
#train = pd.read_csv(out)
#train.rename(columns = {'_matchId': 'win'}, inplace = True)
#train['win'] = train['win'].map(lambda x: label_dict[x])
target = 'win'
# if "Set" not in train.columns:
#     train["Set"] = np.random.choice(["train", "valid"], p =[.9, .1], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index
test_indices = train[train.Set=="test"].index

In [9]:
#test = pd.read_csv(test_out)
#train.rename(columns = {'_matchId': 'win'}, inplace = True)
#train['win'] = train['win'].map(lambda x: label_dict[x])
#test

In [10]:
# train = pd.read_csv(out)
# label = pd.read_csv(label_path)
# train = pd.concat([train, label], axis=1)
# train.rename(columns = {'win_100': 'win'}, inplace = True)
# target = 'win'
# if "Set" not in train.columns:
#     train["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))
# train_indices = train[train.Set=="train"].index
# valid_indices = train[train.Set=="valid"].index
# test_indices = train[train.Set=="test"].index
# train

# Simple preprocessing

Label encode categorical features and fill empty cells.

In [11]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)

win 2
Set 3


In [12]:
categorical_columns

['win', 'Set']

In [13]:
categorical_dims

{'win': 2, 'Set': 3}

In [14]:
# check that pipeline accepts strings
train.loc[train[target]==0, target] = "lose"
train.loc[train[target]==1, target] = "win"

# Define categorical features for categorical embeddings

In [15]:
unused_feat = ['Set']

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


# Network parameters

In [16]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":10, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'sparsemax' # "sparsemax, entmax"
                }

clf = TabNetClassifier(**tabnet_params
                      )

/opt/ml/tabnet/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


# Training

In [17]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [18]:
max_epochs = 1000 if not os.getenv("CI", False) else 2

In [ ]:
# This illustrates the warm_start=False behaviour
save_history = []
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['accuracy'],
        max_epochs=max_epochs , patience=50,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    )
    save_history.append(clf.history["valid_accuracy"])
    
assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 0.69681 | train_accuracy: 0.51064 | valid_accuracy: 0.51151 |  0:00:20s
epoch 1  | loss: 0.69216 | train_accuracy: 0.52488 | valid_accuracy: 0.52527 |  0:00:40s
epoch 2  | loss: 0.69139 | train_accuracy: 0.52216 | valid_accuracy: 0.52073 |  0:01:00s
epoch 3  | loss: 0.68997 | train_accuracy: 0.53888 | valid_accuracy: 0.53789 |  0:01:21s
epoch 4  | loss: 0.68844 | train_accuracy: 0.5433  | valid_accuracy: 0.54512 |  0:01:40s
epoch 5  | loss: 0.68813 | train_accuracy: 0.54447 | valid_accuracy: 0.54432 |  0:02:00s
epoch 6  | loss: 0.68723 | train_accuracy: 0.5482  | valid_accuracy: 0.54966 |  0:02:19s
epoch 7  | loss: 0.68705 | train_accuracy: 0.54954 | valid_accuracy: 0.55105 |  0:02:40s
epoch 8  | loss: 0.68649 | train_accuracy: 0.54909 | valid_accuracy: 0.55004 |  0:03:00s
epoch 9  | loss: 0.68614 | train_accuracy: 0.54998 | valid_accuracy: 0.54969 |  0:03:19s
epoch 10 | loss: 0.68564 | train_accuracy: 0.55212 | valid_accuracy: 0.55394 |  0:03:40s
epoch 11 | loss: 0.68

In [ ]:
# plot losses
plt.plot(clf.history['loss'])

In [ ]:
# plot acc
plt.plot(clf.history['train_accuracy'])
plt.plot(clf.history['valid_accuracy'])

In [ ]:
# plot learning rates
plt.plot(clf.history['lr'])

## Predictions

In [ ]:
preds = clf.predict_proba(X_test)
test_acc = accuracy_score(y_true=[0 if x =='lose' else 1 for x in y_test ], y_pred=[0 if x < 0.5 else 1 for x in preds[:,1]])

preds_valid = clf.predict_proba(X_valid)
valid_acc = accuracy_score(y_true=[0 if x =='lose' else 1 for x in y_valid ], y_pred=[0 if x < 0.5 else 1 for x in preds_valid[:,1]])
print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_acc}")

In [ ]:
# check that best weights are used
assert np.isclose(valid_acc, np.max(clf.history['valid_accuracy']), atol=1e-6)

In [ ]:
clf.predict(X_test)

# Save and load Model

In [ ]:
# save tabnet model
saving_path_name = "./tabnet_model_test_1"
saved_filepath = clf.save_model(saving_path_name)

In [ ]:
# define new model with basic parameters and load state dict weights
loaded_clf = TabNetClassifier()
loaded_clf.load_model(saved_filepath)

In [ ]:
loaded_preds = loaded_clf.predict_proba(X_test)
loaded_test_acc = accuracy_score(y_true=[0 if x =='lose' else 1 for x in y_test ], y_pred=[0 if x < 0.5 else 1 for x in loaded_preds[:,1]])
print(f"FINAL TEST SCORE FOR {dataset_name} : {loaded_test_acc}")

In [ ]:
assert(test_acc == loaded_test_acc)

In [ ]:
loaded_clf.predict(X_test)

# Global explainability : feat importance summing to 1

In [ ]:
clf.feature_importances_.shape

# Local explainability and masks

In [ ]:
explain_matrix, masks = clf.explain(X_test)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20,20))

for i in range(3):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")


# XGB

In [ ]:
#pip install xgboost

In [ ]:
from xgboost import XGBClassifier

clf_xgb = XGBClassifier(max_depth=8,
    learning_rate=0.1,
    n_estimators=1000,
    verbosity=0,
    silent=None,
    objective='binary:logistic',
    booster='gbtree',
    n_jobs=-1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None,)

clf_xgb.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        early_stopping_rounds=200,
        verbose=10)

In [ ]:
preds = np.array(clf_xgb.predict_proba(X_valid))
valid_acc = accuracy_score(y_true= [0 if x =='lose' else 1 for x in y_valid ], y_pred=[0 if x < 0.5 else 1 for x in preds[:,1]])
print(valid_acc)

preds = np.array(clf_xgb.predict_proba(X_test))
test_acc = accuracy_score(y_true=[0 if x =='lose' else 1 for x in y_test ], y_pred=[0 if x < 0.5 else 1 for x in preds[:,1]])
print(test_acc)